In [25]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer,OrdinalEncoder
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import classification_report, confusion_matrix
import xgboost as xgb
import shap

np.random.seed(42)

In [26]:
DATA_PATH = os.path.join(os.getcwd(),"..", "data", "raw", "churn-data.csv")
df=pd.read_csv(DATA_PATH)

In [27]:
df.drop(columns=['RowNumber', 'CustomerId', 'Surname'], inplace=True)

In [28]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


# feature engineering based on previous analysis

In [29]:

class FeatureEngineering:
    def __init__(self, df):
        self.df = df.copy()

    # Age groups mapping
    def age_group(self, age):
        if age < 30:
            return "Young"
        elif age < 45:
            return "Adult"
        elif age < 60:
            return "Senior"
        else:
            return "Elderly"

    def make_feature_extraction(self):
        df = self.df.copy()

        # -------------------------
        # 1. Zero balance indicator
        # -------------------------
        df["IsZeroBalance"] = (df["Balance"] == 0).astype(int)

        # -------------------------
        # 2. Age groups
        # -------------------------
        df["AgeGroup"] = df["Age"].apply(self.age_group)

        # -------------------------
        # 3. Credit score tiers
        # -------------------------
        df["CreditTier"] = pd.cut(
            df["CreditScore"],
            bins=[200, 580, 670, 740, 800, 900],
            labels=["Poor", "Fair", "Good", "VeryGood", "Excellent"],
            include_lowest=True
        )

        # -------------------------
        # 4. Customer Value (improved)
        # Balance + Salary is better indicator
        # -------------------------
        df["CustomerValue"] = df["Balance"] + df["EstimatedSalary"]

        # -------------------------
        # 5. Age × NumOfProducts
        # -------------------------
        df["AgeProduct"] = df["Age"] * df["NumOfProducts"]

        # -------------------------
        # 6. Activity Score 
        # -------------------------
        df["ActivityScore"] = df["IsActiveMember"] * df["NumOfProducts"]

        # -------------------------
        # 7. Log Balance/Salary Ratio (BEST version)
        # More stable, avoids skew
        # -------------------------
        df["LogBalanceSalaryRatio"] = (
            np.log1p(df["Balance"]) - np.log1p(df["EstimatedSalary"])
        )

        # -------------------------
        # 8. High balance flag
        # -------------------------
        df["HighBalance"] = (df["Balance"] > df["Balance"].median()).astype(int)

        return df
fe = FeatureEngineering(df)
df_new_features = fe.make_feature_extraction()
df_new_features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 19 columns):
 #   Column                 Non-Null Count  Dtype   
---  ------                 --------------  -----   
 0   CreditScore            10000 non-null  int64   
 1   Geography              10000 non-null  object  
 2   Gender                 10000 non-null  object  
 3   Age                    10000 non-null  int64   
 4   Tenure                 10000 non-null  int64   
 5   Balance                10000 non-null  float64 
 6   NumOfProducts          10000 non-null  int64   
 7   HasCrCard              10000 non-null  int64   
 8   IsActiveMember         10000 non-null  int64   
 9   EstimatedSalary        10000 non-null  float64 
 10  Exited                 10000 non-null  int64   
 11  IsZeroBalance          10000 non-null  int64   
 12  AgeGroup               10000 non-null  object  
 13  CreditTier             10000 non-null  category
 14  CustomerValue          10000 non-null  

In [30]:
X = df_new_features.drop('Exited', axis=1)
y = df_new_features['Exited']

#split the data into training and testing sets with stratification
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

# check shape of the splits
print("df shape:", df_new_features.shape)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

df shape: (10000, 19)
X_train shape: (8000, 18)
X_test shape: (2000, 18)
y_train shape: (8000,)
y_test shape: (2000,)


In [31]:
# Define feature categories
numerical_features = ['CreditScore', 'Tenure','Age', 'Balance', 'EstimatedSalary','LogBalanceSalaryRatio','CustomerValue']
skewed_numerical_features=["AgeProduct"]
categorical_features = ['AgeGroup', 'CreditTier']
ordeinal_features=['Geography', 'Gender']
ready_cols = list(set(X_train.columns.tolist()) - set(numerical_features) - set(categorical_features)- set(skewed_numerical_features) - set(ordeinal_features))

In [32]:
len(X_train.columns.tolist())

18

In [33]:
len(ready_cols)+len(numerical_features)+len(skewed_numerical_features)+len(categorical_features)+len(ordeinal_features)

18

In [34]:
# check skewness of numerical features
skewed_feats = X_train[numerical_features].skew().sort_values(ascending=False)
print("Skewness of numerical features:\n", skewed_feats)    

Skewness of numerical features:
 Age                      1.035706
Tenure                   0.013124
EstimatedSalary          0.007126
CustomerValue           -0.074331
CreditScore             -0.079540
Balance                 -0.141721
LogBalanceSalaryRatio   -0.538151
dtype: float64


# pipline

In [35]:
# for non skewed numerical features
num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
])

# for skewed numerical features
skewed_num_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('log_transform', FunctionTransformer(np.log1p, validate=False))
    ])


# for categorical features
ordeinal_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('ordinal', OrdinalEncoder(categories=[
        ['Young', 'Adult', 'Senior', 'Elderly'],  # AgeGroup
        ['Poor', 'Fair', 'Good', 'VeryGood', 'Excellent']  # CreditTier
    ]))
])

#for orderinal features
cat_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore',sparse_output=False, drop='first'))
])


#for ready to use features
ready_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))
])
# Combine all pipelines into a ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_pipeline, numerical_features),
        ('skewed_num', skewed_num_pipeline, skewed_numerical_features),
        ('ord', ordeinal_pipeline, categorical_features),
        ('cat', cat_pipeline, ordeinal_features),
        ('ready', ready_pipeline, ready_cols)
    ]
)
#---------------------------------------------------------
# add noise features to test preprocessing robustness
#---------------------------------------------------------
'''
num_noise_features = 2
for i in range(num_noise_features):
    noise_feature_name = f'NoiseFeature_{i+1}'
    X_train[noise_feature_name] = np.random.rand(X_train.shape[0])
    X_test[noise_feature_name] = np.random.rand(X_test.shape[0])
    ready_cols.append(noise_feature_name)

'''
#---------------------------------------------------------
# Fit and transform the training data
#---------------------------------------------------------
X_train_processed = preprocessor.fit_transform(X_train) 
X_test_processed = preprocessor.transform(X_test)
print("Processed X_train shape:", X_train_processed.shape)
print("Processed X_test shape:", X_test_processed.shape)



Processed X_train shape: (8000, 19)
Processed X_test shape: (2000, 19)


In [36]:
#compute class weights to handle class imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
print("Class weights:", class_weights_dict)

Class weights: {0: np.float64(0.6279434850863422), 1: np.float64(2.4539877300613497)}


In [39]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(k_neighbors=5, sampling_strategy=0.7,random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_processed, y_train)
print("Resampled X_train shape:", X_train_resampled.shape)
print("Resampled y_train shape:", y_train_resampled.shape)

Resampled X_train shape: (10829, 19)
Resampled y_train shape: (10829,)


In [41]:
#compute class weights to handle class imbalance
class_weights = compute_class_weight(class_weight='balanced', classes=np.unique(y_train_resampled), y=y_train_resampled)
class_weights_dict = {i : class_weights[i] for i in range(len(class_weights))}
print("Class weights:", class_weights_dict)

Class weights: {0: np.float64(0.85), 1: np.float64(1.2142857142857142)}


In [42]:
# xgboost model with class weights
xgb_model = xgb.XGBClassifier(n_estimators=60, class_weight=class_weights_dict)
xgb_model.fit(X_train_resampled, y_train_resampled)
y_pred = xgb_model.predict(X_test_processed)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:13:13] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "class_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Confusion Matrix:
 [[1523   70]
 [ 204  203]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.96      0.92      1593
           1       0.74      0.50      0.60       407

    accuracy                           0.86      2000
   macro avg       0.81      0.73      0.76      2000
weighted avg       0.85      0.86      0.85      2000



In [ ]:
# xgboost model with class weights
xgb_model = xgb.XGBClassifier(n_estimators=60, class_weight=class_weights_dict)
xgb_model.fit(X_train_processed, y_train)
y_pred = xgb_model.predict(X_test_processed)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Confusion Matrix:
 [[1516   77]
 [ 206  201]]
Classification Report:
               precision    recall  f1-score   support

           0       0.88      0.95      0.91      1593
           1       0.72      0.49      0.59       407

    accuracy                           0.86      2000
   macro avg       0.80      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000



/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:07:51] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "class_weight" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


In [49]:
import optuna
from sklearn.metrics import f1_score
import xgboost as xgb

def objective(trial):
    # XGBoost hyperparameters
    params = {
        "objective": "binary:logistic",
        "eval_metric": "logloss",
        "use_label_encoder": False,

        # Class imbalance handling
        "scale_pos_weight": class_weights_dict[1] / class_weights_dict[0],

        # Trial suggestions
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "max_depth": trial.suggest_int("max_depth", 3, 12),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10)
    }

    model = xgb.XGBClassifier(**params)
    model.fit(X_train_resampled, y_train_resampled)

    preds = model.predict(X_test_processed)
    f1 = f1_score(y_test, preds)

    return f1


# Create and run study
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=200, show_progress_bar=True)

# Best results
print("Best F1 Score:", study.best_value)
print("Best Hyperparameters:", study.best_params)

# Train final model with best parameters
best_params = study.best_params
best_params["objective"] = "binary:logistic"
best_params["eval_metric"] = "logloss"
best_params["use_label_encoder"] = False
best_params["scale_pos_weight"] = class_weights_dict[1] / class_weights_dict[0]



[I 2025-11-25 21:17:31,809] A new study created in memory with name: no-name-30609b70-7854-4010-9f0f-afbc982c045e
  0%|          | 0/200 [00:00<?, ?it/s]/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:17:31] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
Best trial: 0. Best value: 0.580559:   0%|          | 1/200 [00:01<05:02,  1.52s/it]/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:17:33] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
Best trial: 1. Best value: 0.603825:   1%|          | 2/200 [00:01<02:26,  1.35it/s]

[I 2025-11-25 21:17:33,330] Trial 0 finished with value: 0.5805592543275633 and parameters: {'n_estimators': 184, 'max_depth': 10, 'learning_rate': 0.21965677428856123, 'subsample': 0.5436732406123927, 'colsample_bytree': 0.8528195282979216, 'gamma': 0.35827294730098713, 'min_child_weight': 6}. Best is trial 0 with value: 0.5805592543275633.
[I 2025-11-25 21:17:33,522] Trial 1 finished with value: 0.6038251366120219 and parameters: {'n_estimators': 147, 'max_depth': 8, 'learning_rate': 0.08143675248238609, 'subsample': 0.8272617020945641, 'colsample_bytree': 0.7909035699237714, 'gamma': 0.1942413883665861, 'min_child_weight': 8}. Best is trial 1 with value: 0.6038251366120219.


/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:17:33] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
Best trial: 2. Best value: 0.616798:   1%|          | 2/200 [00:01<02:26,  1.35it/s]/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:17:33] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-11-25 21:17:33,612] Trial 2 finished with value: 0.6167979002624672 and parameters: {'n_estimators': 93, 'max_depth': 5, 'learning_rate': 0.17026162961987956, 'subsample': 0.9205933550763556, 'colsample_bytree': 0.9972794459902337, 'gamma': 2.86732739775596, 'min_child_weight': 7}. Best is trial 2 with value: 0.6167979002624672.


Best trial: 3. Best value: 0.625166:   2%|▏         | 4/200 [00:02<01:19,  2.45it/s]/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:17:33] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


...

[I 2025-11-25 21:18:39,051] Trial 196 finished with value: 0.6355140186915887 and parameters: {'n_estimators': 171, 'max_depth': 12, 'learning_rate': 0.08723417573855982, 'subsample': 0.5941431150374656, 'colsample_bytree': 0.880874754015203, 'gamma': 4.5236567401110355, 'min_child_weight': 10}. Best is trial 184 with value: 0.6453333333333333.


Best trial: 184. Best value: 0.645333:  99%|█████████▉| 198/200 [01:07<00:00,  2.29it/s]/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:18:39] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[I 2025-11-25 21:18:39,733] Trial 197 finished with value: 0.6345381526104418 and parameters: {'n_estimators': 248, 'max_depth': 11, 'learning_rate': 0.11744228078685824, 'subsample': 0.6541092910251148, 'colsample_bytree': 0.9298796537968516, 'gamma': 4.996736791606425, 'min_child_weight': 1}. Best is trial 184 with value: 0.6453333333333333.


Best trial: 184. Best value: 0.645333: 100%|█████████▉| 199/200 [01:08<00:00,  2.57it/s]/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:18:40] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
Best trial: 184. Best value: 0.645333: 100%|██████████| 200/200 [01:08<00:00,  2.93it/s]

[I 2025-11-25 21:18:40,009] Trial 198 finished with value: 0.6259946949602122 and parameters: {'n_estimators': 255, 'max_depth': 10, 'learning_rate': 0.07994375015732098, 'subsample': 0.6840344317346162, 'colsample_bytree': 0.8944195602640778, 'gamma': 4.377394793674369, 'min_child_weight': 8}. Best is trial 184 with value: 0.6453333333333333.
[I 2025-11-25 21:18:40,167] Trial 199 finished with value: 0.6271409749670619 and parameters: {'n_estimators': 162, 'max_depth': 11, 'learning_rate': 0.10824899092844503, 'subsample': 0.5188566170648856, 'colsample_bytree': 0.8535965906348535, 'gamma': 4.805524146468711, 'min_child_weight': 9}. Best is trial 184 with value: 0.6453333333333333.
Best F1 Score: 0.6453333333333333
Best Hyperparameters: {'n_estimators': 175, 'max_depth': 10, 'learning_rate': 0.08540501426483674, 'subsample': 0.5818224474876816, 'colsample_bytree': 0.8443435747241653, 'gamma': 4.269177007467712, 'min_child_weight': 9}


In [50]:
# xgboost model with class weights
xgb_model = xgb.XGBClassifier(**best_params)
xgb_model.fit(X_train_resampled, y_train_resampled)
y_pred = xgb_model.predict(X_test_processed)
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("test Classification Report:\n", classification_report(y_test, y_pred))

/home/over-mind/Downloads/end-to-end-churn-project/env/lib/python3.12/site-packages/xgboost/training.py:199: UserWarning: [21:18:46] WARNING: /workspace/src/learner.cc:790: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


Confusion Matrix:
 [[1492  101]
 [ 165  242]]
test Classification Report:
               precision    recall  f1-score   support

           0       0.90      0.94      0.92      1593
           1       0.71      0.59      0.65       407

    accuracy                           0.87      2000
   macro avg       0.80      0.77      0.78      2000
weighted avg       0.86      0.87      0.86      2000



In [51]:
y_pred = xgb_model.predict(X_train_resampled)
print("train Classification Report:\n", classification_report(y_train_resampled, y_pred))

train Classification Report:
               precision    recall  f1-score   support

           0       0.91      0.95      0.93      6370
           1       0.92      0.87      0.90      4459

    accuracy                           0.92     10829
   macro avg       0.92      0.91      0.91     10829
weighted avg       0.92      0.92      0.92     10829



# shap based features && threshold tunning  then meta learner arch

In [17]:

#---------------------------------------------------------
# SHAP feature importance
#---------------------------------------------------------
# Create an explainer for the XGBoost model
explainer = shap.Explainer(xgb_model, X_train_processed)

# Calculate SHAP values for the training set
shap_values = explainer(X_train_processed)

# Compute mean absolute SHAP value for each feature
mean_abs_shap = np.abs(shap_values.values).mean(axis=0)

# Get feature names from the ColumnTransformer
feature_names = []

# For numerical features
feature_names.extend(numerical_features)

# For skewed numerical features
feature_names.extend(skewed_numerical_features)

# For ordinal categorical features
feature_names.extend(categorical_features)

# For one-hot encoded categorical features
# Get the feature names from OneHotEncoder
cat_ohe = preprocessor.named_transformers_['cat']['onehot']
cat_ohe_features = cat_ohe.get_feature_names_out(ordeinal_features)
feature_names.extend(cat_ohe_features)

# For ready features (including noise)
feature_names.extend(ready_cols)

# Make a DataFrame for feature importance
shap_df = pd.DataFrame({
    'Feature': feature_names,
    'SHAP_Value': mean_abs_shap
})

# Sort by importance
shap_df = shap_df.sort_values(by='SHAP_Value', ascending=False).reset_index(drop=True)

# Print in desired format
print("Important Features:==================================================")
for idx, row in shap_df.iterrows():
    print(f"{idx+1}. {row['Feature']:<25} : {row['SHAP_Value']:.4f}")


100%|===================| 7996/8000 [00:34<00:00]        

Important Features:==================================================
1. Age                       : 0.6027
2. NumOfProducts             : 0.4103
3. ActivityScore             : 0.2817
4. AgeProduct                : 0.1785
5. Gender_Male               : 0.1708
6. LogBalanceSalaryRatio     : 0.1523
7. Geography_Germany         : 0.1445
8. Balance                   : 0.1107
9. CustomerValue             : 0.0856
10. CreditScore               : 0.0848
11. EstimatedSalary           : 0.0607
12. Tenure                    : 0.0491
13. AgeGroup                  : 0.0298
14. Geography_Spain           : 0.0256
15. IsActiveMember            : 0.0186
16. HasCrCard                 : 0.0152
17. HighBalance               : 0.0031
18. CreditTier                : 0.0007
19. IsZeroBalance             : 0.0000


# 🎯 Gold Rule: “Better 7 useful features than 19 useless ones.”